# EDA on household columns / index calculation

This notebook will investigate columns for the household data in cambodia to find the relevant column for index calculation

In [1]:
import sys
sys.path.append("../../")

from relativewealth import settings
from relativewealth.dhs import DHSDataManager
from geowrangler import dhs

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
dhsmanager = DHSDataManager()

In [4]:
# Set country-specific variables
country_config = {
    'Philippines': {
        'country_osm':'philippines',
        'ookla_year': 2019,
        'nightlights_year' : 2017,
        'country_code': 'ph',
        'dhs_household_dta_path' : settings.DATA_DIR/"dhs/ph/PHHR71DT/PHHR71FL.DTA",
        'dhs_geographic_shp_path' : settings.DATA_DIR/"dhs/ph/PHGE71FL/PHGE71FL.shp"
    },
    'Timor Leste': {
        'country_osm':'east-timor',
        'ookla_year': 2019,
        'nightlights_year' : 2016,
        'country_code': 'tl',
        'dhs_household_dta_path' : settings.DATA_DIR/"dhs/tl/TLHR71DT/TLHR71FL.DTA",
        'dhs_geographic_shp_path' : settings.DATA_DIR/"dhs/tl/TLGE71FL/TLGE71FL.shp"
    },
    'Cambodia': {
        'country_osm':'cambodia',
        'ookla_year': 2019,
        'nightlights_year' : 2014,
        'country_code': 'kh',
        'dhs_household_dta_path' : settings.DATA_DIR/"dhs/kh/KHHR73DT/KHHR73FL.DTA",
        'dhs_geographic_shp_path' : settings.DATA_DIR/"dhs/kh/KHGE71FL/KHGE71FL.shp"
    },
    'Myanmar': {
        'country_osm':'myanmar',
        'ookla_year': 2019,
        'nightlights_year' : 2015,
        'country_code': 'mm',
        'dhs_household_dta_path' : settings.DATA_DIR/"dhs/mm/MMHR71DT/MMHR71FL.DTA",
        'dhs_geographic_shp_path' : settings.DATA_DIR/"dhs/mm/MMGE71FL/MMGE71FL.shp"
    }
}

## Load Cambodia Data Without Applying GW config

In [6]:
dhs_kh = dhs.load_dhs_file(country_config["Cambodia"]["dhs_household_dta_path"])

In [16]:
kh_cols_with_drinking_water = [x for x in dhs_kh.columns if "drinking water" in x]
kh_cols_with_drinking_water

['na - source of drinking water',
 'na - source of non-drinking water',
 'source of drinking water during the dry season',
 'same source of drinking water during wet and dry season',
 'source of drinking water during wet season']

In [17]:
dhs_kh[kh_cols_with_drinking_water].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15825 entries, 0 to 15824
Data columns (total 5 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   na - source of drinking water                            0 non-null      float64
 1   na - source of non-drinking water                        0 non-null      float64
 2   source of drinking water during the dry season           15820 non-null  float64
 3   same source of drinking water during wet and dry season  15823 non-null  float64
 4   source of drinking water during wet season               15811 non-null  float64
dtypes: float64(5)
memory usage: 741.8 KB


## Investigate the GW config

From this cell, it looks like GW converts `na - source of drinkign water` as its drinking water column, but we know that this column has no values!

In [11]:
dhs.load_column_config('kh')

{'cluster number': 'DHSCLUST',
 'wealth index factor score (5 decimals)': 'Wealth Index',
 'country code and phase': 'country code and phase',
 'number of rooms used for sleeping': 'rooms',
 'has electricity': 'electric',
 'has mobile telephone': 'mobile telephone',
 'has radio': 'radio',
 'has television': 'television',
 'has car/truck': 'car/truck',
 'has refrigerator': 'refrigerator',
 'has motorcycle/scooter': 'motorcycle',
 'main floor material': 'floor',
 'type of toilet facility': 'toilet',
 'na - source of drinking water': 'drinking water'}

In [12]:
dhs_kh['na - source of drinking water']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
15820   NaN
15821   NaN
15822   NaN
15823   NaN
15824   NaN
Name: na - source of drinking water, Length: 15825, dtype: float64

Using `same source of drinking water during wet and dry season` might be a better choice, but the values are binary (maybe it means yes/no?)

In [19]:
dhs_kh['same source of drinking water during wet and dry season'].value_counts()

1.0    10916
0.0     4907
Name: same source of drinking water during wet and dry season, dtype: int64

Looking at 'source of drinking water during the dry season' and 'source of drinking water during wet season' seems to make more sense

In [25]:
dhs_kh['source of drinking water during the dry season']

0        43.0
1        43.0
2        51.0
3        51.0
4        43.0
         ... 
15820    43.0
15821    31.0
15822    32.0
15823    32.0
15824    43.0
Name: source of drinking water during the dry season, Length: 15825, dtype: float64

In [24]:
dhs_kh['source of drinking water during wet season']

0        51.0
1        43.0
2        51.0
3        51.0
4        51.0
         ... 
15820    31.0
15821    31.0
15822    32.0
15823    32.0
15824    51.0
Name: source of drinking water during wet season, Length: 15825, dtype: float64

In [29]:
dhs_kh[['source of drinking water during the dry season', 'source of drinking water during wet season', 'same source of drinking water during wet and dry season']]

,source of drinking water during the dry season,source of drinking water during wet season,same source of drinking water during wet and dry season
0,43.0,51.0,0.0
1,43.0,43.0,1.0
2,51.0,51.0,1.0
3,51.0,51.0,1.0
4,43.0,51.0,0.0
...,...,...,...
15820,43.0,31.0,0.0
15821,31.0,31.0,1.0
15822,32.0,32.0,1.0
15823,32.0,32.0,1.0


## Sense check with PH

In [26]:
dhs_ph = dhs.load_dhs_file(country_config["Philippines"]["dhs_household_dta_path"])

In [27]:
ph_cols_with_drinking_water = [x for x in dhs_ph.columns if "drinking water" in x]
ph_cols_with_drinking_water

['source of drinking water', 'source of non-drinking water']

In [28]:
dhs_ph['source of drinking water']

0        11
1        71
2        71
3        13
4        11
         ..
27491    13
27492    42
27493    41
27494    12
27495    41
Name: source of drinking water, Length: 27496, dtype: int8